# Формулировка задания

Задача написать приложение/скрипт на языке Python, который будет анализировать ссылки в основном блоке статьи на википедии и находить верную цепочку перехода, что б пройти от url1 =*> url2 или наоборот.

Ограничения:

1) Обе входные ссылки будут на одном языке. И вы должны находить переходы на статьи на том же языке. (Можно на русском, английском)

2) Ссылки должны быть или из тела статьи или из блока References. Учитываются ссылки только на википедию.

3) Нужно реализовать rate-limit и передавать его 3 параметром - нельзя создавать подключений больше чем limit

4) Если за 5 переходов цель не достигнута url1 -> url2 и url2 -> url1 - сообщаем об этом. Не учитываем ссылки глубиной более 5 и избегаем дубликатов

Вход (пример): https://en.wikipedia.org/wiki/Six_degrees_of_separation https://en.wikipedia.org/wiki/American_Broadcasting_Company 10

Выход: url2 =>[url] => [url] =>url1

RateLimiter:

https://pypi.org/project/ratelimiter/

https://akshayranganath.github.io/Rate-Limiting-With-Python/

# Импорт библиотек

In [2]:
!pip install ratelimiter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import requests
import numpy as np
import re
from bs4 import BeautifulSoup
from ratelimiter import RateLimiter
import time
import queue
import copy

# Вычисление пути

In [8]:
def findWikiPath(startUrl, endUrl, maxDepth, rateLimit):
  
  #функция определения языка ссылки Википедии
  def getLinkLang(link):
    return re.search(r'(https://)(.*?)(.wiki)', link).group(2)

  #проверяем, что языки у входной и выходной ссылки одинаковы
  LANG_PREFIX = getLinkLang(startUrl)

  if (LANG_PREFIX != getLinkLang(endUrl)):
    print('The languages of the pages do not match.')
    return

  #функция определяет, ведет ли ссылка на Википедию
  def isWikiLink(link):
    return re.search('(https://).*(wiki).*(/wiki/)', link)

  def limitedCb(until):
    duration = int(round(until - time.time()))
    print('Rate limited, sleeping for {:d} seconds'.format(duration))

  #функция получает разметку
  @RateLimiter(max_calls=rateLimit, period=20, callback=limitedCb)
  def getPageContent(url):
    try:
      page = requests.get(url)

      #'тернарный оператор' здесь читается справа-налево
      return ('', page) [page.status_code == 200]

    except Exception as e:
      print('Cannot get content from {link}\n')
      return ''

  #функция делает полную ссылку
  def makeFullLink(link):
    if not re.search('https', link):
      return 'https://' + LANG_PREFIX + '.wikipedia.org' + link
    else:
      return link


  #функция получает ссылки на английском на Википедию
  #из тела и блока References
  #id="bodyContent" - id тела
  #ol class = references - список ссылок из References
  def getLinksFromWiki(url):

    page = getPageContent(url)

    if not (page):
      return []

    soup = BeautifulSoup(page.text, 'html.parser')
    body = soup.find(id="bodyContent")
    references = soup.find_all("ol", {"class": "references"})

    if (references):
      references = references[len(references)-1]

    if (body): 
      bodyA = body.find_all('a')
    else:
      bodyA = []
    
    if (references):
      refsA = references.find_all('a')
    else:
      refsA = []

    foundRefs = bodyA + refsA

    links = []

    if (foundRefs):
      for a in foundRefs:
        if (a):
          ref = a.get('href')
          if (ref):
            if not (re.search('#cite_', ref)):
              usefulLink = makeFullLink(ref)
              if (isWikiLink(usefulLink)) and (getLinkLang(usefulLink) == LANG_PREFIX):
                links.append(usefulLink)

    return links


  startLink = startUrl
  endLink = endUrl

  #очереди текущего и следующего уровня глубины
  currentLinks = queue.Queue()
  nextLinks = queue.Queue()

  currentDepth = 0
  nextLinks.put(startLink)

  #словарь, в котором храняется посещенные ссылки
  visitedLinks = {startLink: ''}

  while not nextLinks.empty():

    #следующий уровень глубины
    currentDepth = currentDepth + 1
    print(f'Depth: {currentDepth}')

    #следующие ссылки становятся текущими
    currentLinks.queue = copy.copy(nextLinks.queue)

    #очищаем место для новых ссылок
    nextLinks.queue.clear()
    
    #перебираем текущие ссылки
    while not currentLinks.empty():
      currentLink = currentLinks.get()
      print(f'current depth = {currentDepth}, parse {currentLink} \n')

      for nextLink in getLinksFromWiki(currentLink):

        #если нашли искомую ссылку
        if (nextLink == endLink):
          path = [endLink, currentLink]
          link = visitedLinks[currentLink]

          #восстанавливаем путь исходной страницы
          while link:
            path.append(link)
            link = visitedLinks[link]

          print('Path found:')

          #разворачиваем список
          print('\n'.join(path[::-1]))

          #print('visitedLinks')
          #for key, value in visitedLinks.items():
          #  print("{0}: {1}".format(key,value))

          return

        #если это не нужная нам ссылка, но мы 
        #еще проходили по этой ссылке и не достигли maxDepth
        if (nextLink not in visitedLinks) and (currentDepth != maxDepth):
          #добавляем ссылку в посещенные
          visitedLinks[nextLink] = currentLink
          nextLinks.put(nextLink)

  print(f'Path from {startLink} to {endLink} with depth {maxDepth} was not found.')

# Примеры работы функции

In [9]:
start = 'https://en.wikipedia.org/wiki/Six_degrees_of_separation'
end = 'https://en.wikipedia.org/wiki/American_Broadcasting_Company'

rateLimit = 10
maxDepth = 5

findWikiPath(start, end, maxDepth, rateLimit)

Depth: 1
current depth = 1, parse https://en.wikipedia.org/wiki/Six_degrees_of_separation 

Path found:
https://en.wikipedia.org/wiki/Six_degrees_of_separation
https://en.wikipedia.org/wiki/American_Broadcasting_Company


In [10]:
start = 'https://ru.wikipedia.org/wiki/Nightwish#%D0%98%D1%81%D0%BF%D0%BE%D0%BB%D0%BD%D1%8F%D0%B2%D1%88%D0%B8%D0%B5%D1%81%D1%8F_%D1%82%D0%BE%D0%BB%D1%8C%D0%BA%D0%BE_%D0%B2%D0%B6%D0%B8%D0%B2%D1%83%D1%8E'
end = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D0%BB%D0%BB%D0%B8%D0%B2%D1%83%D0%B4%D1%81%D0%BA%D0%B0%D1%8F_%C2%AB%D0%90%D0%BB%D0%BB%D0%B5%D1%8F_%D1%81%D0%BB%D0%B0%D0%B2%D1%8B%C2%BB'

rateLimit = 10
maxDepth = 5

findWikiPath(start, end, maxDepth, rateLimit)

Depth: 1
current depth = 1, parse https://ru.wikipedia.org/wiki/Nightwish#%D0%98%D1%81%D0%BF%D0%BE%D0%BB%D0%BD%D1%8F%D0%B2%D1%88%D0%B8%D0%B5%D1%81%D1%8F_%D1%82%D0%BE%D0%BB%D1%8C%D0%BA%D0%BE_%D0%B2%D0%B6%D0%B8%D0%B2%D1%83%D1%8E 

Depth: 2
current depth = 2, parse https://ru.wikipedia.org/wiki/%D0%92%D0%B8%D0%BA%D0%B8%D0%BF%D0%B5%D0%B4%D0%B8%D1%8F:%D0%9F%D0%B0%D1%82%D1%80%D1%83%D0%BB%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5 

current depth = 2, parse https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%B9%D0%BB:Nightwish_wordmark.svg 

current depth = 2, parse https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%B9%D0%BB:Nightwish_au_Rockhal_2015.JPG 

current depth = 2, parse https://ru.wikipedia.org/wiki/%D0%AD%D1%88-%D1%81%D1%8E%D1%80-%D0%90%D0%BB%D1%8C%D0%B7%D0%B5%D1%82%D1%82 

current depth = 2, parse https://ru.wikipedia.org/wiki/%D0%9B%D1%8E%D0%BA%D1%81%D0%B5%D0%BC%D0%B1%D1%83%D1%80%D0%B3 

current depth = 2, parse https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%BC

In [11]:
start = 'https://ru.wikipedia.org/wiki/%D0%9E%D0%B3%D1%83%D1%80%D0%B5%D1%86_%D0%BE%D0%B1%D1%8B%D0%BA%D0%BD%D0%BE%D0%B2%D0%B5%D0%BD%D0%BD%D1%8B%D0%B9'
end = 'https://ru.wikipedia.org/wiki/%D0%A0%D1%8F%D0%B4_%D0%A4%D1%83%D1%80%D1%8C%D0%B5'


rateLimit = 20
maxDepth = 2

findWikiPath(start, end, maxDepth, rateLimit)

Depth: 1
current depth = 1, parse https://ru.wikipedia.org/wiki/%D0%9E%D0%B3%D1%83%D1%80%D0%B5%D1%86_%D0%BE%D0%B1%D1%8B%D0%BA%D0%BD%D0%BE%D0%B2%D0%B5%D0%BD%D0%BD%D1%8B%D0%B9 

Depth: 2
current depth = 2, parse https://ru.wikipedia.org/wiki/%D0%9E%D0%B3%D1%83%D1%80%D0%B5%D1%86_(%D0%B7%D0%BD%D0%B0%D1%87%D0%B5%D0%BD%D0%B8%D1%8F) 

current depth = 2, parse https://ru.wikipedia.org/wiki/%D0%9E%D0%B3%D1%83%D1%80%D1%86%D1%8B_(%D0%B7%D0%BD%D0%B0%D1%87%D0%B5%D0%BD%D0%B8%D1%8F) 

current depth = 2, parse https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%B9%D0%BB:Cucumis_sativus20090812_496.jpg 

current depth = 2, parse https://ru.wikipedia.org/wiki/%D0%91%D0%B8%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D0%BA%D0%B0 

current depth = 2, parse https://ru.wikipedia.org/wiki/Eukaryota 

current depth = 2, parse https://ru.wikipedia.org/wiki/Plantae 

current depth = 2, parse https://ru.wikipedia.org/wiki/Viridiplantae 

curre